In [1]:
from konlpy.tag import *
import datetime
from IPython.display import display

import re
import numpy as np
import pandas as pd
from pprint import pprint
import mecab
mecab = mecab.MeCab()

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
import pyLDAvis
import pyLDAvis.gensim
import string

from multiprocessing import Pool
import time
import numpy as np

import preprocessing_py as prep
import multiprocessing



In [55]:
it_df = pd.read_csv('preprocessing_culture.csv')
it_df.shape

(107, 3)

In [56]:
it_df = it_df.iloc[:,1:]
it_df.rename(columns={'0':'raw','1':'tuned'},inplace=True)
# it_df['tuned'] = it_df['tuned'].apply(lambda x : x.replace(",",' '))

In [57]:
it_df.tail()

,raw,tuned
102,"""[서울신문]에버랜드 제공에버랜드가 개학·입학·입사 등 새로운 출발을 앞둔 사람들을...","서울,신문,에버랜드,제공,에버랜드,개학,입학,입사,새로운,출발,사람,응원,축제,연,..."
103,"""경남수목원 10여 그루 중 한 그루, 풍년화도 피어 봄 재촉납매 개화[경남도산림환...","경남,수목원,여,그루,중,그루,풍년,화도,피어,봄,재촉,개화,경남,산림,환경,연구원..."
104,"""(창원=연합뉴스) 경남수목원에 봄을 재촉하는 풍년화가 꽃을 피워 눈길을 끌고 있다...","창,원,연합,경남,수목원,봄,재촉,풍년,화가,꽃,눈길,끌,경남,산림,환경,연구원,제..."
105,"""(창원=연합뉴스) 경남수목원에 봄 전령사로 통하는 납매(臘梅)가 꽃을 피워 이른 ...","창,원,연합,경남,수목원,봄,전령,사,통,꽃,봄,소식,전하,경남,산림,환경,연구원,..."
106,"""(창원=연합뉴스) 경남수목원에 봄 전령사로 통하는 납매(臘梅)가 꽃을 피워 이른 ...","창,원,연합,경남,수목원,봄,전령,사,통,꽃,봄,소식,전하,경남,산림,환경,연구원,..."


In [58]:
tuned_data = it_df['tuned'].tolist()
tuned_data = [[i] for i in tuned_data]

In [61]:
word_dict = corpora.Dictionary(documents)

In [71]:
print(word_dict)

Dictionary(2621 unique tokens: ['가성', '간격', '감촉', '개', '결제']...)


In [63]:
doc_term_matrix = [word_dict.doc2bow(doc) for doc in documents]
print(doc_term_matrix[:1])

33
[[(0, 1), (1, 1), (2, 1), (3, 4), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 2), (24, 2), (25, 1), (26, 1), (27, 1), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 3), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 3), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 3), (82, 1), (83, 2), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1)]]


In [64]:
def return_lda(doc_term_matrix,dictionary,num_topic_params) :
# Creating the object for LDA model using gensim library
    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(
        corpus=doc_term_matrix,
        id2word=dictionary, 
        num_topics = num_topic_params, 
        distributed = False,
        random_state=100,
        update_every=1,
        passes = 10,
        alpha='auto',
        eta = None,
        per_word_topics=True,
        chunksize=100)

    return ldamodel

_____

In [65]:
model = return_lda(doc_term_matrix,word_dict,5)
model.show_topic(1)

[('문화재', 0.010208644),
 ('교류', 0.008848238),
 ('직', 0.008819905),
 ('문화', 0.008193457),
 ('분야', 0.00750284),
 ('사업', 0.0068446337),
 ('항공', 0.0068220864),
 ('설명', 0.0061531),
 ('의원', 0.005657046),
 ('손', 0.005624949)]

In [66]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model,doc_term_matrix,word_dict)

In [67]:
pyLDAvis.gensim.prepare(model,doc_term_matrix,word_dict)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.172036 -0.062265       1        1  33.984470
4      0.124547 -0.134496       2        1  23.586151
3      0.047683  0.117156       3        1  20.250647
1     -0.031479  0.032463       4        1  13.319360
2      0.031285  0.047142       5        1   8.859374, topic_info=     Category       Freq Term      Total  loglift  logprob
term                                                      
1818  Default  22.000000   영화  22.000000  30.0000  30.0000
1351  Default  27.000000   한글  27.000000  29.0000  29.0000
1071  Default  53.000000   의원  53.000000  28.0000  28.0000
1064  Default  59.000000    손  59.000000  27.0000  27.0000
2023  Default  15.000000   북한  15.000000  26.0000  26.0000
885   Default  20.000000   제국  20.000000  25.0000  25.0000
194   Default  24.000000   교류  24.000000  24.0000  24.0000
611   Default  63.000000  박물관  63.000000  23.0000  23.0000
597   Default  19.000000   문명  19.000000  22.0000  22.0000
1395  Default  26.000000   나전  26.000000  21.0000  21.0000
1185  Default  21.000000  문화재  21.000000  20.0000  20.0000
1505  Default  26.000000   칠기  26.000000  19.0000  19.0000
1928  Default  15.000000   맑음  15.000000  18.0000  18.0000
1494  Default  10.000000    직  10.000000  17.0000  17.0000
134   Default   9.000000   부산   9.000000  16.0000  16.0000
2029  Default   6.000000   상영   6.000000  15.0000  15.0000
455   Default  37.000000   국립  37.000000  14.0000  14.0000
224   Default  21.000000   문화  21.000000  13.0000  13.0000
1426  Default  14.000000   분야  14.000000  12.0000  12.0000
138   Default  16.000000   사업  16.000000  11.0000  11.0000
857   Default  32.000000   작품  32.000000  10.0000  10.0000
1256  Default  12.000000   여성  12.000000   9.0000   9.0000
2421  Default   7.000000   항공   7.000000   8.0000   8.0000
348   Default  10.000000   인간  10.000000   7.0000   7.0000
1916  Default   9.000000   구름   9.000000   6.0000   6.0000
1986  Default   8.000000   남북   8.000000   5.0000   5.0000
976   Default   9.000000   파괴   9.000000   4.0000   4.0000
1513  Default  13.000000   학예  13.000000   3.0000   3.0000
232   Default   8.000000   봉사   8.000000   2.0000   2.0000
1660  Default   9.000000   감독   9.000000   1.0000   1.0000
...       ...        ...  ...        ...      ...      ...
2105   Topic5   1.196434   허가   1.803639   2.0132  -6.2781
2071   Topic5   1.195711   접촉   1.803816   2.0125  -6.2787
2083   Topic5   1.195738   차원   1.803895   2.0125  -6.2787
2009   Topic5   1.195472   매개   1.804017   2.0122  -6.2789
2020   Topic5   1.194853   봄날   1.804215   2.0116  -6.2794
2046   Topic5   1.194742   역대   1.804257   2.0115  -6.2795
2002   Topic5   1.194341   동무   1.804326   2.0111  -6.2799
2093   Topic5   1.194301   통일   1.804418   2.0110  -6.2799
2065   Topic5   1.193891   장면   1.804527   2.0106  -6.2802
134    Topic5   6.073560   부산   9.635807   1.9622  -4.6535
1838   Topic5   2.278458   초청   3.642128   1.9546  -5.6339
908    Topic5   2.282254   준비   3.667606   1.9493  -5.6323
1997   Topic5   2.281857   당시   3.728056   1.9328  -5.6325
200    Topic5   2.822206   국제   4.928949   1.8661  -5.4199
2097   Topic5   2.820218   평양   5.391330   1.7757  -5.4206
194    Topic5   8.254155   교류  24.496197   1.3359  -4.3467
822    Topic5   2.274997   이해   4.494496   1.7428  -5.6355
57     Topic5   3.369591   이후   9.087196   1.4316  -5.2427
1660   Topic5   3.371096   감독   9.335535   1.4051  -5.2422
1510   Topic5   2.279512    편   5.289737   1.5819  -5.6335
510    Topic5   3.375182    년  10.315201   1.3065  -5.2410
3      Topic5   2.294903    개   6.871503   1.3270  -5.6268
224    Topic5   3.919058   문화  21.907513   0.7027  -5.0916
53     Topic5   2.842139    위  13.266617   0.8830  -5.4129
1493   Topic5   2.828844  지난해  14.861502   0.7648  -5.4176
324    Topic5   2.282915    북   7.608255   1.2199  -5.6320
1986   Topic5   2.286512   남북   8.064147   1.1633  -5.6304
108    Topic5   2.277302    김   9.05841